In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(
    text_splitter=splitter,
)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(
    docs,
    cached_embeddings,
)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            ---------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']

    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )

map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question create a final answer. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            --------------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {
    "context": map_chain,
    "question": RunnablePassthrough(),
} | final_prompt | llm

chain.invoke("Describe Victory Mansions")
# chain.invoke("Where does Winston go to work?")


AIMessage(content='Victory Mansions is a run-down apartment building in London, the chief city of Airstrip One in George Orwell\'s novel "1984." The building is dilapidated, with peeling paint, broken elevators, and flickering lights. The hallway smells of boiled cabbage and old rag mats, and a colored poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU" is tacked to the wall. The flat is located seven flights up, with a faulty lift that rarely works. The building lacks basic amenities and is part of an economy drive in preparation for Hate Week, with the electric current being cut off during daylight hours. Surrounding Victory Mansions are grimy landscapes, rotting nineteenth-century houses, bombed sites with rubble, and sordid colonies of wooden dwellings. The building symbolizes the oppressive and impoverished living conditions under the totalitarian regime of the Party in the novel.')